# Covid-19 Vaccines in California and their Impacts

In [ ]:
#Hypothesis:
#1.	College towns had an impact on Covid-19 surges in 2020.
#2.	Vaccines have directly impacted California’s case and death rates.


In [2]:
import pandas as pd
import scipy.stats as sts
import matplotlib.pyplot as plt
import numpy as np

### Cleaning Data

In [ ]:
# Cleaning Cases Data

In [ ]:
# Cleaning Vaccinations Data

In [ ]:
# Cleaning Hospital Data

In [6]:
# Cleaning Colleges Data
college_data = pd.read_csv('Data/ca_colleges.csv')

college_data.columns = ['School_ID', 'School_name', 'Year', 'Duplicate', 'Street_Address', 'City', 'Zip_code', 
                        'County', 'Longitude', 'Latitude', 'Total_Enrollment', 'Full_time_enrollment', 
                        'Part_time_enrollment']

college_data.drop(columns=['School_ID', 'Year', 'Duplicate', 'Street_Address', 'City', 'Zip_code', 
                        'County', 'Full_time_enrollment', 
                        'Part_time_enrollment'], inplace=True)

college_data.dropna(subset=['Total_Enrollment'], inplace=True)
college_data.head()

,School_name,Longitude,Latitude,Total_Enrollment
0,Academy of Art University,-122.400578,37.787943,9812.0
1,Academy of Chinese Culture and Health Sciences,-122.269839,37.805972,133.0
3,Avalon School of Cosmetology-Alameda,-122.243566,37.764203,47.0
4,College of Alameda,-122.279303,37.781017,5667.0
5,Allan Hancock College,-120.421144,34.943716,11894.0


### Connecting API

### Summary Statistics

## Research Analysis

### 1.	How has the introduction of the Covid vaccine affected case/death rate in California?

### 2.	Do “College” towns play a larger role in case surges?

### 3.	Are there “Vaccination Deserts”? What impact have Covid-19 and the vaccines had in these areas?

### 4.	Can we predict when California will return to “normal”?